In [14]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [15]:
from tensorflow.keras.layers import Input, Conv1D
from tensorflow.keras import Model
import numpy as np

In [16]:
inputs = Input(shape=(20,3))
x = Conv1D(2,4,3)(inputs)
model = Model(inputs, x)

In [17]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 20, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 6, 2)           │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26 (104.00 B)

 Trainable params: 26 (104.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.weights

[<KerasVariable shape=(4, 3, 2), dtype=float32, path=conv1d_1/kernel>,
 <KerasVariable shape=(2,), dtype=float32, path=conv1d_1/bias>]

In [19]:
X = np.random.rand(1,20,3)
X

array([[[0.69317084, 0.27514949, 0.72261989],
        [0.4004044 , 0.4367038 , 0.35932463],
        [0.08412116, 0.56992585, 0.94290655],
        [0.93057819, 0.54074314, 0.28227966],
        [0.71447469, 0.79410571, 0.4100902 ],
        [0.12512464, 0.83206553, 0.45888015],
        [0.88072123, 0.30830401, 0.18738574],
        [0.25369066, 0.54013257, 0.34599301],
        [0.05920863, 0.83139686, 0.37209592],
        [0.01096649, 0.99577125, 0.96489828],
        [0.93697957, 0.85376491, 0.83263996],
        [0.450299  , 0.37488993, 0.83785345],
        [0.71892225, 0.20328651, 0.84388803],
        [0.81824413, 0.76966768, 0.88540278],
        [0.06328908, 0.1796773 , 0.67402022],
        [0.4347067 , 0.73348192, 0.93302255],
        [0.56729644, 0.71308706, 0.9517126 ],
        [0.9679539 , 0.52835951, 0.59704553],
        [0.40476777, 0.23386184, 0.06484938],
        [0.71647166, 0.00596706, 0.66473633]]])

In [20]:
y = model.predict(X)
y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


array([[[-0.31858763, -0.01445049],
        [-0.60669386, -0.4150122 ],
        [-0.07068481, -1.1251775 ],
        [-1.034431  , -0.01323686],
        [-0.2522466 , -0.5591998 ],
        [-0.9952065 , -0.16519947]]], dtype=float32)

In [21]:
X_in = [[int(X[0][i][j]*2**10) for j in range(3)] for i in range(20)]
weights = [[[int(model.weights[0].numpy()[i][j][k]*2**10) for k in range(2)] for j in range(3)] for i in range(4)]
bias = [int(model.weights[1].numpy()[i]*2**20) for i in range(2)]
    

In [22]:
def Conv1DInt(nInputs, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[str(input[i][j]) for j in range(nChannels)] for i in range(nInputs)]
    Weights = [[[str(weights[i][j][k]) for k in range(nFilters)] for j in range(nChannels)] for i in range(kernelSize)]
    Bias = [str(bias[i]) for i in range(nFilters)]
    out = [[0 for _ in range(nFilters)] for j in range((nInputs - kernelSize)//strides + 1)]
    for i in range((nInputs - kernelSize)//strides + 1):
        for j in range(nFilters):
            for k in range(kernelSize):
                for l in range(nChannels):
                    out[i][j] += input[i*strides + k][l]*weights[k][l][j]
            out[i][j] += bias[j]
            out[i][j] = str(out[i][j] // n)
    return Input, Weights, Bias, out

In [23]:
X_in, weights, bias, out = Conv1DInt(20, 3, 2, 4, 3, 2**10, X_in, weights, bias)
out

[['-325', '-15'],
 ['-621', '-424'],
 ['-72', '-1150'],
 ['-1057', '-13'],
 ['-257', '-571'],
 ['-1017', '-170']]

In [24]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out
}

In [25]:
import json

In [26]:
with open("conv1D_input.json", "w") as f:
    json.dump(in_json, f)